In [ ]:
# MANUAL tSNE on MNIST dataset: 
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
from tqdm import tqdm

In [8]:
# mnist = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
# import tensor
%pip install tensorflow
import tensorflow as tf
%pip install tensorflow-datasets
import tensorflow_datasets as tfds

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\koola\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\koola\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   - -------------------------------------- 0.3/5.3 MB ? eta -:--:--
   --------- ------------------------------ 1.3/5.3 MB 2.9 MB/s eta 0:00:02
   ------------- -------------------------- 1.8/5.3 MB 2.9 MB/s eta 0:00:02
   ----------------- ---------------------- 2.4/5.3 MB 2.9 MB/s eta 0:00:02
   ----------------------- ---------------- 3.1/5.3 MB 3.1 MB/s eta 0:00:01
   --------------------------- ------------ 3.7/5.3 MB 3.1 MB/s eta 0:00:01
   --------------------------------- ------ 4.5/5.3 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------  5.2/5.3 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 5.3/5.3 MB 3.0 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21

In [ ]:
from scipy.spatial.distance import cdist

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [9]:
# pseudocode: 
mnist = tfds.load('mnist', split='train', as_supervised=False) # load mnist dataset from tensorflow datasets
mnist = mnist.shuffle(buffer_size=10000) # shuffle the dataset to get random samples
# load mnist into matrix X
x = mnist.data.numpy().reshape(-1, 28*28)
x = x.astype(np.float64)
y = mnist.targets.numpy()

# sample n images 
n = 4000
np.random.seed(0)
idx = np.random.choice(len(x), n)
x = x[idx]
print(x.shape)

# print(x)
# prep X: 0-1 range;
# x = x / 255
# x = x - x.mean(axis=0)
# print(np.mean(x, axis=0))
# print(np.mean(x, axis=1))
xx = x - np.min(x)
xx = xx / np.max(xx) # min-max normalization
xx = xx - np.mean(xx,axis=0)
# print(xx)
# run PCA to 50 dimensions
x = xx
pca = PCA(n_components=50)
x_pca = pca.fit_transform(x)

print(x_pca.shape)
# compute pairwise distance matrix D

D = cdist(x_pca, x_pca)

# compute similarities p_ij for each row i:
P = np.zeros([n, n])
for i in range(n):
    beta = 0.6
    D_i = D[i, :]
    P_i = np.zeros(n)
    # binary like search for beta(i)
    trials = 0
    hdiff = 1
    betamin = -np.inf
    betamax = np.inf
    while(np.abs(hdiff) > 1e-5 and trials < 50):
        P_i = np.exp(-D_i * beta)
        P_i_sum = np.sum(P_i)
        # betac = (betamin + betamax) / 2
        # h = np.sum(np.exp(-D_i * P_i_sum))
        h = np.log(P_i_sum) + beta * (np.dot(D_i, P_i)/P_i_sum)
        hdiff = h - np.log2(30)
        if hdiff > 0:
            # betac = (betamin + betamax) / 2
            betamin = beta
            if(betamax == np.inf):
                beta = beta * 2
            else:
                beta = (beta + betamax) / 2
        else:
            # betac = (betamin + betamax) / 2
            betamax = beta
            if(betamin == -np.inf):
                beta = beta / 2
            else:
                beta = (beta + betamin) / 2
    trials += 1
    # beta = betac
    P[i, :] = P_i  

#make sure P is correct
for i in range(n):
    P[i,i] = 1e-5

P = (P + P.T) / 2 # Symmetric
P = P / np.sum(P) # Normalized
# print(P[1,0])

print(P.shape)
constKL = np.sum(P * np.log(P))
print("ConstKL: ", constKL)
# initialize tSNE
max_iter = 451
epsilon = 500
min_gain = 0.01
Y = np.random.randn(n, 2)
# dY = np.zeros((n, 2))
iY = np.zeros([n, 2])
gains = np.ones([n, 2])

# run tSNE
for i in tqdm(range(max_iter)):
    # compute Q
    Y2 = Y**2
    sum_Y2 = np.sum(Y2, axis=1, keepdims=True)
    Q = np.zeros([n, n])
    
    dist = 1.0 / (sum_Y2 - 2 * np.dot(Y, Y.T) + sum_Y2.T)
    # Q = 1 / (1 + dist)
    np.fill_diagonal(dist, 1e-5)
    Q = dist / np.sum(dist)
    # Q = np.maximum(Q, 1e-12)
    
    # compute dY
    PQ = (P - Q) * dist

    # for k in range(n):
    #     dY[k] = np.sum(np.tile(PQ[:, k] * Q[:, k], (2, 1)).T * (Y[k] - Y), axis=0)
    dY = 4 * np.dot(np.diag(np.sum(PQ, axis=1)) - PQ, Y)
    
    # perform the update
    # if i == 0:
    #     gains = np.sign(dY) != np.sign(iY)
    # else:
    #     gains[gains > 20] = 20
    #     gains[gains < 0.01] = 0.01

    # iY = epsilon * iY - 0.2 * gains * dY
    iY = -epsilon * dY
    Y = Y + iY
    # Y = Y - np.tile(np.mean(Y, axis=0), (n, 1))
    Y = Y - np.mean(Y, axis=0)
    
    # compute the KL divergence
    if i % 50 == 0:
        C = constKL - np.sum(P * np.log(Q))
        print("Iteration", i, "KL divergence is", C)
        plt.scatter(Y[:, 0], Y[:, 1], c=y[idx], cmap='tab10')
        plt.xlim(-50, 50)
        plt.ylim(-50, 50)
        plt.show()

    

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\koola\tensorflow_datasets\mnist\incomplete.YM9TDL_3.0.1\mnist-train.tfrecord*...:   0%|    …

KeyboardInterrupt: 